In [1]:
print('Hello World!')

Hello World!


In [1]:
#WEB SCRAPPER TO EXTRACT TITLE ND ABSTRACTS AND DOI FROM PUBMED. AND THEN DOWNLOAD PDFS FROM SCIHUB

#Step 1 - GO TO PUBMED WEBSITE and search for the citation
#Step 2 - Go to the search results
#Step 4 - Access the paper and extract the title abstract and doi as required
#Step 4 - Append it to the dataframe. Save it
#Step 5 - Using the extracted doi, search for the paper on scihub
#Step 6 - Download the paper
#Step 7 - Append names of the pdf files to the dataframe
#Step 8 - Save the dataframe

In [17]:
import pandas as pd
import re

def remove_brackets(text):
    """
    Remove square brackets from a text string but keep the contents inside the brackets.

    Parameters:
    text (str): The input string containing square brackets.

    Returns:
    str: The input string with only square brackets removed.
    """
    # Use a regular expression to remove only the square brackets
    cleaned_text = re.sub(r'[\[\]]', '', text)
    return cleaned_text


df = pd.read_excel(r'UVH- Screening (First stage).xlsx', sheet_name='DB searching')
df = df[['sr', 'Journal', 'Citation', 'Source', 'f_selection', 'f_rejection',
       'Type_of_exclusion', 'reason_for_exclsuion', 'Comment',
       'final_selection']]

df.Citation = df.Citation.apply(lambda x: remove_brackets(x))

In [22]:
df.Source.value_counts()

Source
PubMed            359
Scopus            138
Embase             85
Web of Science     67
Pubmed             12
Name: count, dtype: int64

In [23]:
pubmed = df[(df['Source'] == 'PubMed') | (df['Source'] == 'Pubmed')]
# Replace NaNs in the 'final_selection' column with 0
pubmed['final_selection'] = pubmed['final_selection'].fillna(0)
pubmed['f_selection'] = pubmed['f_selection'].fillna(0)


/var/folders/cl/6yq4kf4j44n6g477yhn772hr0000gn/T/ipykernel_8246/1267788284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed['final_selection'] = pubmed['final_selection'].fillna(0)
/var/folders/cl/6yq4kf4j44n6g477yhn772hr0000gn/T/ipykernel_8246/1267788284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed['f_selection'] = pubmed['f_selection'].fillna(0)


## Extract titles abstracts and doi

In [26]:
from scrapper_pubmed import Scrapper
citations = list(pubmed.Citation)
bot = Scrapper(headless=False)
titles,abstracts,doi = bot.start(citations)

Scraping Information:   0%|          | 0/371 [00:00<?, ?it/s]

In [27]:
pubmed['titles'] = titles
pubmed['abstracts']= abstracts
pubmed['dois']=doi

/var/folders/cl/6yq4kf4j44n6g477yhn772hr0000gn/T/ipykernel_8246/2186184840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed['titles'] = titles
/var/folders/cl/6yq4kf4j44n6g477yhn772hr0000gn/T/ipykernel_8246/2186184840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed['abstracts']= abstracts
/var/folders/cl/6yq4kf4j44n6g477yhn772hr0000gn/T/ipykernel_8246/2186184840.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [28]:
pubmed

,sr,Journal,Citation,Source,f_selection,f_rejection,Type_of_exclusion,reason_for_exclsuion,Comment,final_selection,titles,abstracts,dois
2,3,Ann Otol Rhinol Laryngol Suppl,S. M. Abel and H. O. Barber Measurement of opt...,PubMed,0.0,1.0,Outcome,Not included symptoms,NaN,0.0,Measurement of optokinetic nystagmus for otone...,Abstract\nOptokinetic nystagmus was recorded a...,doi: 10.1177/00034894810901s201.
4,5,Eur Arch Otorhinolaryngol,"G. O. Acar, E. Acioğlu, O. Enver, C. Ar and S....",PubMed,0.0,1.0,Population,NaN,NaN,0.0,Unilateral sudden hearing loss as the first si...,Abstract\nChronic myeloid leukemia (CML) is on...,doi: 10.1007/s00405-007-0382-1.
6,7,Rev Laryngol Otol Rhinol (Bord),"W. Adjibabi, F. Avakoudjo, O. Biaou, S. L. Afo...",PubMed,0.0,1.0,Population,Unilateral nasal obstruction,NaN,0.0,[Seven cases of inverted nasosinusal papilloma...,Abstract\nObjective: Sinonasal inverted papill...,Not Found
8,9,J Clin Neurosci,"S. H. Ahn, J. E. Shin and C. H. Kim Final diag...",PubMed,1.0,NaN,NaN,NaN,NaN,0.0,Final diagnosis of patients with clinically su...,Abstract\nVestibular neuritis is one of the mo...,doi: 10.1016/j.jocn.2017.02.064.
9,10,J Surg Case Rep,"Y. Aladham, O. Ahmed, S. A. S. Hassan and E. F...",PubMed,0.0,1.0,Population,Superior SSC dehissance,NaN,0.0,Traumatic superior semicircular canal dehiscen...,Abstract\nSuperior semicircular canal dehiscen...,doi: 10.1093/jscr/rjaa592.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,655,Hippocampus,"Y. Zheng, C. L. Darlington and P. F. Smith Imp...",PubMed,0.0,1.0,Population,Animal study (rats),NaN,0.0,Impairment and recovery on a food foraging tas...,Abstract\nIt has been suggested that the vesti...,doi: 10.1002/hipo.20149.
655,656,PLoS One,"Y. J. Zhou, J. Yu, Y. Z. Wu, L. Tian, Z. Han, ...",PubMed,0.0,1.0,Population,Mumps,NaN,0.0,The potential dysfunction of otolith organs in...,Abstract\nObjective: To investigate the relati...,doi: 10.1371/journal.pone.0181907.
658,659,Nervenarzt,"A. Zwergal, V. Kirsch, J. Gerb, J. Dlugaiczyk,...",PubMed,0.0,1.0,Design,Not related to subject,NaN,0.0,[Neuro-otology: at the borders of ear and brain],Abstract\nVertigo and dizziness are frequent c...,doi: 10.1007/s00115-018-0598-x.
659,660,Brain Struct Funct,"A. Zwergal, J. Schlichtiger, G. Xiong, R. Beck...",PubMed,0.0,1.0,Design,Review,NaN,0.0,Sequential [(18)F]FDG µPET whole-brain imaging...,Abstract\nUnilateral inner ear damage is follo...,doi: 10.1007/s00429-014-0899-1.


In [29]:
len(pubmed[pubmed.titles == 'Not Found'])

10

In [30]:
pubmed.to_csv('pubmed_abstracts.csv')
#pubmed.to_excel('pubmed_abstracts.xlsx')

In [2]:
import pandas as pd
pubmed = pd.read_csv('pubmed_abstracts.csv')

## Download PDFs

In [2]:
from download_scihub import SciHubDownloader
downloader = SciHubDownloader(headless=False)
dois = list(pubmed.dois)
dois = [x.replace("doi: ","").strip() for x in dois]
pdfs = downloader.download_paper(dois)

Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.
Button not found on the page.


In [3]:
pubmed['pdfs'] = pdfs

In [6]:
pubmed[pubmed.pdfs == "No pdf"]

,Unnamed: 0,sr,Journal,Citation,Source,f_selection,f_rejection,Type_of_exclusion,reason_for_exclsuion,Comment,final_selection,titles,abstracts,dois,pdfs
2,6,7,Rev Laryngol Otol Rhinol (Bord),"W. Adjibabi, F. Avakoudjo, O. Biaou, S. L. Afo...",PubMed,0.0,1.0,Population,Unilateral nasal obstruction,NaN,0.0,[Seven cases of inverted nasosinusal papilloma...,Abstract\nObjective: Sinonasal inverted papill...,Not Found,No pdf
12,17,18,Arch Ital Biol,"P. Andre, O. Pompeiano, D. Manzoni and P. D'As...",PubMed,0.0,1.0,Population,Animal study,NaN,0.0,Muscarinic receptors in the cerebellar vermis ...,Abstract\n1. The Purkinje (P)-cells of the cer...,Not Found,No pdf
19,37,38,Int Tinnitus J,"S. Barozzi, F. Di Berardino, E. Arisi and A. C...",PubMed,1.0,NaN,NaN,NaN,NaN,0.0,A comparison between oculomotor rehabilitation...,Abstract\nRehabilitation therapy is proved to ...,Not Found,No pdf
20,40,41,J Int Adv Otol,"M. Bassiouni, H. Olze and P. Arens Bilateral H...",PubMed,0.0,1.0,Population,Bilateral loss,NaN,0.0,Bilateral Hearing Loss Due to Metastatic Gastr...,Abstract\nIntracranial metastases of stomach c...,doi: 10.5152/iao.2020.8415.,No pdf
21,41,42,Iran J Otorhinolaryngol,A. Bayat and N. Saki Effects of Vestibular Reh...,PubMed,1.0,NaN,NaN,NaN,In the full text there are symptoms,0.0,Effects of Vestibular Rehabilitation Intervent...,Abstract\nIntroduction: Although vestibular re...,Not Found,No pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,613,614,Ear Hear,"S. J. Wang, W. J. Weng, F. S. Jaw and Y. H. Yo...",PubMed,0.0,1.0,Population,Healthy subjects,NaN,0.0,Not Found,Not Found,Not Found,No pdf
357,633,634,Can Fam Physician,"V. Wu, E. A. Sykes, M. M. Beyea, M. T. W. Simp...",PubMed,0.0,1.0,Design,Review,NaN,0.0,Approach to Ménière disease management,Abstract\nObjective: To provide family physici...,Not Found,No pdf
362,648,649,Zh Nevrol Psikhiatr Im S S Korsakova,M. V. Zamergrad and O. S. Levin Metavestibular...,PubMed,0.0,1.0,Design,Not related to subject,NaN,0.0,[Metavestibular disorders and disorders of hig...,"Abstract in English, Russian\nVertigo, instabi...",doi: 10.17116/jnevro20171176211-15.,No pdf
363,650,651,Acta Otorhinolaryngol Ital,"D. Zanetti, N. Civiero, C. Balzanelli, M. Toni...",PubMed,1.0,NaN,NaN,NaN,NaN,0.0,Improvement of vestibular compensation by Levo...,Abstract\nL-sulpiride is the levorotatory enan...,Not Found,No pdf


In [4]:
pubmed.to_csv('pubmed_pdfs.csv')
#pubmed.to_excel('pubmed_pdfs.xlsx')